# <center>Informatique tc3 (Projet Web) - TD2</center>

## <center style="color: #66d">Base de données SQLite, Firefox et SQLite manager, utilisation via Python</center>

### 1. Prise en main d'une base de données SQLite

Dans le cadre de ce TD nous utiliserons une base de données et développerons du code Python pour enregistrer des données, les modifier, les relire, et les réutiliser pour obtenir des représentations graphiques.

La base de données sera au format SQLite _(fichier .sqlite)._ Toutes les opérations sur une base de données de ce type peuvent être effectuées depuis Python via le module _sqlite3_.

Les fichiers qui vous ont été fournis pour ce TD comportent une base de données nommée _raytracing.sqlite_. Pour en observer le contenu il nous faut un outil. Ce logiciel nous permettra également d'interagir avec le contenu de la base <i>(créer, supprimer ou modifier des tables et des enregistrements, effectuer des requêtes SQL...)</i>.
<img src="FF-SQLite.png" style="float: right; border: 1px solid #CCC; margin-right: 10px">
<p>
L'outil retenu s'appelle SQLite Manager et se présente sous la forme d'un add-on pour le navigateur Web Firefox. Firefox possède lui-même l'insigne avantage de pouvoir s'installer aussi bien sous Windows, que Mac ou Linux.


__Q1. Installation de Firefox et SQLite manager :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Télécharger et installer <a href="https://www.mozilla.org/fr/firefox/new/">Firefox</a> s'il n'est pas déjà sur votre poste, puis installer l'add-on <a href="https://addons.mozilla.org/fr/firefox/addon/sqlite-manager/">SQLite Manager</a> depuis Firefox lui-même.
<p>
Démarrer SQLite Manager et ouvrir la base de données _raytracer.sqlite_ pour constater l'existence d'une table nommée _scene_.
<p>
<img src="SQLite.png">
</div>

### 2. Sérialisation des objets d'une scène de raytracing

La table _scene_ a été conçue pour enregistrer les caractéristiques de scènes du type de celles que vous avez créées lors du TD n°1. Elle comporte les champs nécessaires pour :
<ul style="margin-top:0">
<li style="margin-top:0"> donner un identifiant unique à chacune des scènes enregistrées, sous forme d'un entier _id,_
<li style="margin-top:0"> nommer la scène, via le champ _name,_
<li style="margin-top:0"> indiquer les dimensions de l'image grâce à des entiers _width_ et _height,_
<li style="margin-top:0"> se souvenir du temps de calcul qui a été nécessaire pour obtenir la scène avec le réel _ptime,_
<li style="margin-top:0"> retrouver le fichier grâce à son nom : _filename._
</ul>

Il y a également un champ textuel nommé _serial_ destiné à donner la description de la scène. Cette description doit comporter tous les éléments permettant de reconstruire une scène identique. Ceci inclut les paramètres de la scène, ainsi que la liste de tous les objets qui la constituent, ainsi que leurs paramètres.

La technique permettant d'obtenir une représentation textuelle à partir d'objets en mémoire, s'appelle la __sérialisation__. Nous serons bien entendu également intéressés par l'opération inverse permettant de reconstruire un objet à partir de sa description : la __désérialisation__.

Il existe sous Python des modules comme <a href="https://docs.python.org/3.6/library/pickle.html">pickle</a> permettant d'effectuer ces opérations. Malheureusement, ces modules ne sérialisent pas au format texte, mais dans un format binaire non lisible tel quel par des humains (ce n'est pas drôle), et parfois incompatible d'une version de Python à une autre (on ne peut pas désérialiser un objet sérialisé avec une version différente).

Pour ces raisons, ainsi que pour des raisons pédagogiques, nous sérialiserons nos objets au format <a href="http://www.json.org/">JSON</a>, implémenté en Python via le module <a href="https://docs.python.org/3.6/library/json.html">json</a>.

In [1]:
import json

obj = {"a": 1, "b": 2, "c": 3}
serialized = json.dumps(obj)

# serialized est une chaîne de caractères :
print(type(serialized))

# et contient la description de l'objet :
print(serialized)

<class 'str'>
{"a": 1, "b": 2, "c": 3}


### 2.1 Attributs de base

Pour sérialiser les attributs, chacune des classes du raytracer sera munie d'une méthode _dict()_ qui renverra un dictionnaire avec la liste des attributs à sérialiser. Le fait de renvoyer un dictionnaire et non une chaîne de caractères résultat de la sérialisation JSON, facilitera la sérialisation d'attributs qui doivent être eux-mêmes sérialisés _(cf. ci-dessous "2.3 Composition"),_ ainsi que le chaînage de la sérialisation des classes enfants vers les classes parentes _(cf. 2.4 Héritage.)_

In [2]:
class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)
    
    def dict(self):
        return self.__dict__
    
v = vec3(1,2,3)
serialized = json.dumps(v.dict())
print("v: {}".format(serialized))

v: {"x": 1, "y": 2, "z": 3}


Ceci dit, cela n'est pas forcément toujours aussi simple, et l'on peut tricher en renommant les attributs :

In [3]:
class rgb(vec3):    
    def dict(self):
        return {"r": self.x, "g": self.y, "b": self.z}
    
orange = rgb(1.0, 0.647, 0.0)
serialized = json.dumps(orange.dict())

print("color: {}".format(serialized))

color: {"r": 1.0, "g": 0.647, "b": 0.0}


### 2.2 Classe des objets

Le fait de sérialiser les _objets_ sous forme d'un _dictionnaire_ nous fait perdre une information : le nom de la classe.
C'est la raison pour laquelle nous conviendrons d'ajouter un élément au dictionnaire, dont la clé sera nommée <i>_</i>_class_<i>_</i> et dont la valeur correspondra au nom de la classe de l'objet :

In [11]:
import json
class rgb(vec3):
    def dict(self):
        return {
            "_class_": self.__class__.__name__,
            "r": self.x,
            "g":self.y,
            "b":self.z
        }

orange = rgb(1.0, 0.647, 0.0)
serialized = json.dumps(orange.dict())

print("color: {}".format(serialized))

color: {"_class_": "rgb", "r": 1.0, "g": 0.647, "b": 0.0}


### 2.3 Composition

Dans l'exemple ci-dessous, les attributs _position_ et _color_ de la classe _LightSource_ sont respectivement des objets de la classe _vec3_ et _rgb._

Pour obtenir une représentation sérialisée d'une source de lumière, il faut que la méthode _dict()_ de la classe _LightSource_ renvoie un dictionnaire pour ses attributs _position_ et _color_. L'obtention d'un dictionnaire pour ces attributs s'effectuera bien sûr grâce à leur méthode _dict()._ 

__Q2. Sérialisation d'un objet composé :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
A partir du fragment ci-dessous, écrire le code de la classe _LightSource_ avec le nécessaire pour en permettre la sérialisation, et ajouter le code permettant de tester le résultat. Penser également à redéfinir la classe _vec3_ pour que la sérialisation obtenue pour l'attribut _position_ comporte bien la clé <i>_</i>_class_<i>_</i>.
<p>
<pre style="background-color:#eef; margin:0">
class LightSource():
    def __init__(self,position,color):
        self.position = position
        self.color = color

    def dict(self):
        pass

source = LightSource( vec3(1,2,3), rgb(1,0.5,0))
</pre>
</div>

In [13]:
import json
class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)
    
    def dict(self):
        return {
            "_class_": self.__class__.__name__,
            "x": self.x,
            "y": self.y,
            "z": self.z
        }
    
class LightSource():
    def __init__(self,position,color):
        self.position = position
        self.color = color

    def dict(self):
        return {"position" : self.position.dict(), "color" : self.color.dict()}
    

source = LightSource(vec3(1,2,3), rgb(1,0.5,0))

serialized = json.dumps(source.dict())

print("LightSource: {}".format(serialized))

LightSource: {"position": {"_class_": "vec3", "x": 1, "y": 2, "z": 3}, "color": {"_class_": "rgb", "r": 1, "g": 0.5, "b": 0}}


### 2.4 Héritage

Afin de faciliter la sérialisation des classes filles, la méthode _dict()_ d'une classe mère admettra un paramètre _other_ permettant d'inclure les attributs de la classe fille :

In [15]:
class Artifact():
    def __init__(self, color, ambient = 1.0, specular = 0.5, phong = 70, mirror = 0.5):
        self.color = color
        self.ambient = ambient
        self.specular = specular
        self.phong = phong
        self.mirror = mirror
        
    def dict(self, other={}):
        kwargs = ['ambient', 'specular', 'phong', 'mirror']
        return {
            "_class_": self.__class__.__name__,
            **other,
            "color": self.color.dict(),
            **{k: getattr(self,k) for k in kwargs}
        }

class Sphere(Artifact):
    def __init__(self, center, r, *args, **kwargs):
        Artifact.__init__(self, *args, **kwargs)
        self.c = center
        self.r = r
        
    def dict(self):
        return Artifact.dict(self, {
            "center": self.c.dict(),
            "radius": self.r
        })

__Q3. Sérialisation de classes filles :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
&Eacute;crire le code python permettant de tester la sérialisation d'un sphère telle que définie ci-dessus.
<br><br>
__N.B.__ : il est attendu que le centre de la sphère soit spécifié via un objet de la classe _vec3_, et la couleur d'un artefact via un objet de la classe _rgb._
</div>

In [18]:
import json
sphere = Sphere(vec3(1,2,3),5,rgb(1,1,1),ambient=2, specular= 2, phong=80, mirror= 0.7)

serialized = json.dumps(sphere.dict())

print("Sphere: {}".format(serialized))


Sphere: {"_class_": "Sphere", "center": {"_class_": "vec3", "x": 1, "y": 2, "z": 3}, "radius": 5, "color": {"_class_": "rgb", "r": 1, "g": 1, "b": 1}, "ambient": 2, "specular": 2, "phong": 80, "mirror": 0.7}


## 3. Désérialisation

### 3.1 Désérialisation d'un dictionnaire

Le module _json_ fournit la méthode _loads()_ qui effectue l'opération inverse de _dumps()_ :

In [8]:
import json

obj = {"a": 1, "b": 2, "c": 3}
serialized = json.dumps(obj)
print('serialized : {} {}'.format(type(serialized),serialized))

obj2 = json.loads(serialized)
print('deserialized : {} {}'.format(type(obj2),obj2))

serialized : <class 'str'> {"a": 1, "b": 2, "c": 3}
deserialized : <class 'dict'> {'a': 1, 'b': 2, 'c': 3}


### 3.2 Désérialisation d'un objet

Pour désérialiser un objet il faut appeler le constructeur de la classe ad'hoc avec la liste des paramètres adéquats. Pour cela, munissons nos classes d'une __méthode de classe__ nommée _fromDict()_ qui prendra un dictionnaire en paramètre et renverra l'objet désérialisé.

__N.B.__ Une méthode de classe est indiquée par le décorateur _@classmethod_. Lorsque python appelle une telle méthode, le premier paramètre n'est pas _un objet (i.e. self)_, mais _sa classe_ souvent notée _cls._

In [9]:
class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)
    
    def dict(self):
        return {
            "_class_": self.__class__.__name__,
            **self.__dict__
        }
    
    @classmethod
    def fromDict(cls, _dict):
        return cls(_dict['x'], _dict['y'], _dict['z'])
    
    def __str__(self):
        return 'vec3({},{},{})'.format(self.x,self.y,self.z)

# sérialisation :
v = vec3(1,2,3)
serialized = json.dumps(v.dict())
print('serialized : {} {}'.format(type(serialized),serialized))

# désérialisation
v2 = vec3.fromDict(json.loads(serialized))
print('deserialized : {} {}'.format(type(v2),v2))


serialized : <class 'str'> {"_class_": "vec3", "x": 1, "y": 2, "z": 3}
deserialized : <class '__main__.vec3'> vec3(1,2,3)


## 4. Sérialisation et désérialisation d'une scène

Le module <i>serializable_raytracer</i> joint à ce TD définit les mêmes classes que le module <i>simple_raytracer</i> du TD précédent, mais ajoute les possibilités de sérialisation et de désérialisation décrites ci-dessus. Ainsi, chacune des classes implémente les méthodes _serialize()_ et _deserialize()_ ainsi que _dict()_ et _fromDict()_ qui devraient vous être moins directement utiles, sauf à créer de nouvelles classes.

__N.B. Pour les plus assidus __ qui consultent le code source. Les possibilités décrites ci-dessus ont été implémentées à l'aide d'une classe mère nommée _Serializable_ dont héritent tous les autres objets. Cette classe possède des méthodes génériques conçues de manière à éviter au maximum les redondances de code au sein des classes filles.

__Q4. Sérialisation d'une scène :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une scène à l'aide des classes du module _raytracer,_ puis la sérialiser et afficher le résultat.
<br><br>
Remarquer l'attribut _selector_ des objets bicolores comme _CheckeredPlane_ ou _CheckeredSPhere_. Quelle est sa raison d'être ?
<br>
__N.B.__ la réponse se trouve dans l'énoncé du paragraphe 6. du TD 1.
</div>


SyntaxError: invalid syntax (<ipython-input-19-2209f0fde5ba>, line 1)

Une fois sérialisée, la scène peut être enregistrée en base de données :

In [11]:
import sqlite3

# ouverture d'une connexion avec la base de données
conn = sqlite3.connect('raytracing.sqlite')
c = conn.cursor()

# liste des valeurs enregistrées, dans l'ordre des champs de la table scene
# N.B. on s'attend ici à disposer d'une scène dans la variable "scene" 
data = [
    scene.name,
    scene.serialize(),
    scene.w,
    scene.h
]

# enregistrement des informations par soumission d'une requête SQL INSERT
c.execute("INSERT INTO scene VALUES (NULL, ?, ?, ?, ?, 0, '')", data)
conn.commit()

__Q5. Enregistrement dans la base de données :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Effectuer l'enregistrement et vérifier la présence de l'information dans la base de données via _SQLite Manager_ :
<img src = "SQLite_record.png">
<p>
Préciser à quoi correspondent les valeurs NULL, 0 et '' dans la commande SQL soumise à la base.
</div>

__Q6. Lecture en base de données :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
A l'aide d'une requête SQL <span style="font-family: monospace"><a href="https://sqlite.org/lang_select.html" style="text-decoration: none">SELECT</a></span>, et en vous inspirant du code ci-dessous, récupérer les informations de la scène et afficher le résultat.
<p>
<pre style="background-color:#eef; margin:0">
import sqlite3
conn = sqlite3.connect('raytracing.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM scene WHERE id=1")
info = c.fetchone()</pre></div>

In [12]:
# votre code ici

__Q7. Désérialisation de l'information récupérée et tracé de l'image :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Désérialiser la scène lue en base de données et tracer l'image pour vérifier que l'ensemble de la chaîne sérialisation - enregistrement - lecture - désérialisation s'est bien passée.
</div>

In [13]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_deserialized_scene.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="deserialized_scene.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 7. deserialized_scene.png</i></td>
</tr></table>

__Q8. Mise à jour de l'information en base de données :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Maintenant que le temps de calcul et le nom du fichier sont connus, mettre à jour l'enregistrement en base de données
grâce à une requête SQL <span style="font-family: monospace"><a href="https://sqlite.org/lang_update.html" style="text-decoration: none">UPDATE</a></span> et vérifier le résultat via _SQLite Manager._
<img src="SQLite_deserialized.png">
</div>

In [14]:
# votre code ici

## 5. Couleurs nommées

Il est fastidieux d'être systématiquement obligé de spécifier les couleurs à l'aide de triplets _rgb._ Il serait bien plus pratique d'utiliser leur nom.

### 5.1 Création d'une table en base de données

__Q9. Création d'une table pour les couleurs :__
<div style="background-color:#eef;padding:10px;border-radius:3px">

Dans la base de donnée, créer une table nommée _color_ à l'aide d'une requête SQL <span style="font-family: monospace"><a href="https://www.sqlite.org/lang_createtable.html" style="text-decoration: none">CREATE TABLE</a></span>.
<p>
Cette table devra comporter les champs suivants :
<ul style="margin-top:0">
<li style="margin-top:0"> un identifiant unique nommé _id,_
<li style="margin-top:0"> _name,_ pour le nom de la couleur,
<li style="margin-top:0"> trois champs respectivement nommés _red, green_ et _blue_ pour la valeur de chacune des composantes.
</ul>
<p>
Une fois la table créée, vérifier à l'aide de _SQLite Manager_ que tout s'est bien passé :
<img src="SQLite_color.png">
</div>

In [15]:
# votre code ici

### 5.2 Liste des couleurs nommées

__Q10. Remplissage de la table des couleurs :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
&Agrave; partir du code ci-dessous, remplir la table des couleurs en prenant soin de préciser chacune des composantes à l'aide d'un nombre réel compris entre 0.0 et 1.0, et vérifier le résultat à l'aide de _SQLite Manager_ :
<img src="SQLite_colors.png">
<br><br>
__N.B.__ En cas d'erreur vous pouvez nettoyer la table (i.e. supprimer tout son contenu) via _SQLite Manager_ ou à l'aide de la commande SQL<br>
<code style="background-color:#eef; margin:0">DELETE FROM color</code>
</div>

In [16]:
# Liste des couleurs à insérer dans la base

(name, r, g, b) = ('name','r','g','b')
colors = [
    { name:"aliceblue", r:240, g:248, b:255 },
    { name:"antiquewhite", r:250, g:235, b:215 },
    { name:"aqua", r:0, g:255, b:255 },
    { name:"aquamarine", r:127, g:255, b:212 },
    { name:"azure", r:240, g:255, b:255 },
    { name:"beige", r:245, g:245, b:220 },
    { name:"bisque", r:255, g:228, b:196 },
    { name:"black", r:0, g:0, b:0 },
    { name:"blanchedalmond", r:255, g:235, b:205 },
    { name:"blue", r:0, g:0, b:255 },
    { name:"blueviolet", r:138, g:43, b:226 },
    { name:"brown", r:165, g:42, b:42 },
    { name:"burlywood", r:222, g:184, b:135 },
    { name:"cadetblue", r:95, g:158, b:160 },
    { name:"chartreuse", r:127, g:255, b:0 },
    { name:"chocolate", r:210, g:105, b:30 },
    { name:"coral", r:255, g:127, b:80 },
    { name:"cornflowerblue", r:100, g:149, b:237 },
    { name:"cornsilk", r:255, g:248, b:220 },
    { name:"crimson", r:220, g:20, b:60 },
    { name:"cyan", r:0, g:255, b:255 },
    { name:"darkblue", r:0, g:0, b:139 },
    { name:"darkcyan", r:0, g:139, b:139 },
    { name:"darkgoldenrod", r:184, g:134, b:11 },
    { name:"darkgray", r:169, g:169, b:169 },
    { name:"darkgreen", r:0, g:100, b:0 },
    { name:"darkgrey", r:169, g:169, b:169 },
    { name:"darkkhaki", r:189, g:183, b:107 },
    { name:"darkmagenta", r:139, g:0, b:139 },
    { name:"darkolivegreen", r:85, g:107, b:47 },
    { name:"darkorange", r:255, g:140, b:0 },
    { name:"darkorchid", r:153, g:50, b:204 },
    { name:"darkred", r:139, g:0, b:0 },
    { name:"darksalmon", r:233, g:150, b:122 },
    { name:"darkseagreen", r:143, g:188, b:143 },
    { name:"darkslateblue", r:72, g:61, b:139 },
    { name:"darkslategray", r:47, g:79, b:79 },
    { name:"darkslategrey", r:47, g:79, b:79 },
    { name:"darkturquoise", r:0, g:206, b:209 },
    { name:"darkviolet", r:148, g:0, b:211 },
    { name:"deeppink", r:255, g:20, b:147 },
    { name:"deepskyblue", r:0, g:191, b:255 },
    { name:"dimgray", r:105, g:105, b:105 },
    { name:"dimgrey", r:105, g:105, b:105 },
    { name:"dodgerblue", r:30, g:144, b:255 },
    { name:"firebrick", r:178, g:34, b:34 },
    { name:"floralwhite", r:255, g:250, b:240 },
    { name:"forestgreen", r:34, g:139, b:34 },
    { name:"fuchsia", r:255, g:0, b:255 },
    { name:"gainsboro", r:220, g:220, b:220 },
    { name:"ghostwhite", r:248, g:248, b:255 },
    { name:"gold", r:255, g:215, b:0 },
    { name:"goldenrod", r:218, g:165, b:32 },
    { name:"gray", r:128, g:128, b:128 },
    { name:"grey", r:128, g:128, b:128 },
    { name:"green", r:0, g:128, b:0 },
    { name:"greenyellow", r:173, g:255, b:47 },
    { name:"honeydew", r:240, g:255, b:240 },
    { name:"hotpink", r:255, g:105, b:180 },
    { name:"indianred", r:205, g:92, b:92 },
    { name:"indigo", r:75, g:0, b:130 },
    { name:"ivory", r:255, g:255, b:240 },
    { name:"khaki", r:240, g:230, b:140 },
    { name:"lavender", r:230, g:230, b:250 },
    { name:"lavenderblush", r:255, g:240, b:245 },
    { name:"lawngreen", r:124, g:252, b:0 },
    { name:"lemonchiffon", r:255, g:250, b:205 },
    { name:"lightblue", r:173, g:216, b:230 },
    { name:"lightcoral", r:240, g:128, b:128 },
    { name:"lightcyan", r:224, g:255, b:255 },
    { name:"lightgoldenrodyellow", r:250, g:250, b:210 },
    { name:"lightgray", r:211, g:211, b:211 },
    { name:"lightgreen", r:144, g:238, b:144 },
    { name:"lightgrey", r:211, g:211, b:211 },
    { name:"lightpink", r:255, g:182, b:193 },
    { name:"lightsalmon", r:255, g:160, b:122 },
    { name:"lightseagreen", r:32, g:178, b:170 },
    { name:"lightskyblue", r:135, g:206, b:250 },
    { name:"lightslategray", r:119, g:136, b:153 },
    { name:"lightslategrey", r:119, g:136, b:153 },
    { name:"lightsteelblue", r:176, g:196, b:222 },
    { name:"lightyellow", r:255, g:255, b:224 },
    { name:"lime", r:0, g:255, b:0 },
    { name:"limegreen", r:50, g:205, b:50 },
    { name:"linen", r:250, g:240, b:230 },
    { name:"magenta", r:255, g:0, b:255 },
    { name:"maroon", r:128, g:0, b:0 },
    { name:"mediumaquamarine", r:102, g:205, b:170 },
    { name:"mediumblue", r:0, g:0, b:205 },
    { name:"mediumorchid", r:186, g:85, b:211 },
    { name:"mediumpurple", r:147, g:112, b:219 },
    { name:"mediumseagreen", r:60, g:179, b:113 },
    { name:"mediumslateblue", r:123, g:104, b:238 },
    { name:"mediumspringgreen", r:0, g:250, b:154 },
    { name:"mediumturquoise", r:72, g:209, b:204 },
    { name:"mediumvioletred", r:199, g:21, b:133 },
    { name:"midnightblue", r:25, g:25, b:112 },
    { name:"mintcream", r:245, g:255, b:250 },
    { name:"mistyrose", r:255, g:228, b:225 },
    { name:"moccasin", r:255, g:228, b:181 },
    { name:"navajowhite", r:255, g:222, b:173 },
    { name:"navy", r:0, g:0, b:128 },
    { name:"oldlace", r:253, g:245, b:230 },
    { name:"olive", r:128, g:128, b:0 },
    { name:"olivedrab", r:107, g:142, b:35 },
    { name:"orange", r:255, g:165, b:0 },
    { name:"orangered", r:255, g:69, b:0 },
    { name:"orchid", r:218, g:112, b:214 },
    { name:"palegoldenrod", r:238, g:232, b:170 },
    { name:"palegreen", r:152, g:251, b:152 },
    { name:"paleturquoise", r:175, g:238, b:238 },
    { name:"palevioletred", r:219, g:112, b:147 },
    { name:"papayawhip", r:255, g:239, b:213 },
    { name:"peachpuff", r:255, g:218, b:185 },
    { name:"peru", r:205, g:133, b:63 },
    { name:"pink", r:255, g:192, b:203 },
    { name:"plum", r:221, g:160, b:221 },
    { name:"powderblue", r:176, g:224, b:230 },
    { name:"purple", r:128, g:0, b:128 },
    { name:"red", r:255, g:0, b:0 },
    { name:"rosybrown", r:188, g:143, b:143 },
    { name:"royalblue", r:65, g:105, b:225 },
    { name:"saddlebrown", r:139, g:69, b:19 },
    { name:"salmon", r:250, g:128, b:114 },
    { name:"sandybrown", r:244, g:164, b:96 },
    { name:"seagreen", r:46, g:139, b:87 },
    { name:"seashell", r:255, g:245, b:238 },
    { name:"sienna", r:160, g:82, b:45 },
    { name:"silver", r:192, g:192, b:192 },
    { name:"skyblue", r:135, g:206, b:235 },
    { name:"slateblue", r:106, g:90, b:205 },
    { name:"slategray", r:112, g:128, b:144 },
    { name:"slategrey", r:112, g:128, b:144 },
    { name:"snow", r:255, g:250, b:250 },
    { name:"springgreen", r:0, g:255, b:127 },
    { name:"steelblue", r:70, g:130, b:180 },
    { name:"tan", r:210, g:180, b:140 },
    { name:"teal", r:0, g:128, b:128 },
    { name:"thistle", r:216, g:191, b:216 },
    { name:"tomato", r:255, g:99, b:71 },
    { name:"turquoise", r:64, g:224, b:208 },
    { name:"violet", r:238, g:130, b:238 },
    { name:"wheat", r:245, g:222, b:179 },
    { name:"white", r:255, g:255, b:255 },
    { name:"whitesmoke", r:245, g:245, b:245 },
    { name:"yellow", r:255, g:255, b:0 },
    { name:"yellowgreen", r:154, g:205, b:50 }
]

In [17]:
# votre code ici

### 5.3 Introduction des couleurs dans l'espace global

Nous aimerions maintenant pouvoir utiliser une couleur en s'y référant à partir de son nom, plutôt qu'en instanciant à chaque fois un représentant de la classe _rgb_ à partir de ses composantes.

Pour cela, python possède une fonction nommée _globals()_ qui renvoie le dictionnaire des variables globales, que l'on peut lire pour y retrouver une valeur, ou compléter pour en créer de nouvelles :

In [18]:
x = 33
glob = globals()
print(glob['x'])

glob['y'] = 486
print(y)

33
486


__Q11. Couleurs nommées dans l'espace global :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
&Agrave; partir des considérations ci-dessus, et bien que cela ne soit en général pas considéré comme une bonne pratique de polluer l'espace global avec de nombreuses variables, lire le contenu de la table _color_ et créer une variable pour chacune des couleurs, ainsi qu'un dictionnaire nommé _colors_ permettant d'accéder à l'ensemble d'entre elles.
</div>

In [19]:
# votre code ici

### 5.4 Création d'un module

Pour instancier toute ces couleurs, et éviter si possible de polluer l'espace global, le plus simple est de créer un module nommé <i>raytracer_colors.py</i> avec le code qui vient d'être réalisé.

__N.B.__ Pour cela, il est bon de savoir que la commande "<code>from raytracer\_colors.py import *</code>" importera __toutes les variables__ définies au niveau global au sein de ce module, sauf celles <b>dont le nom commence par un caractère souligné <span>_</span></b>.


__Q12. Module raytracer_colors.py :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer le module <code style="background-color:#eef">raytracer<span>_</span>colors.py</code> en veillant à exporter uniquement les couleurs et le dictionnaire nommé _colors._
</div>

In [20]:
# votre code ici

In [21]:
# Dans cet exemple on ne pollue pas l'espace global
import raytracer_colors as rtc

# Nombre de couleurs dans rtc.colors
print('Raytracer has {} colors'.format(len(rtc.colors.keys())))
print()

# Test de quelques couleurs
print(rtc.salmon, rtc.aquamarine, rtc.colors['forestgreen'])
print()

# Liste des variables exportées par le module
print([c for c in dir(rtc) if not c[0:1] == '_'])

# Pour importer les couleurs dans l'espace global :
from raytracer_colors import *

Raytracer has 147 colors

rgb(0.9765625, 0.5, 0.4453125) rgb(0.49609375, 0.99609375, 0.828125) rgb(0.1328125, 0.54296875, 0.1328125)

['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'colors', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'ligh

### 5.5 Création d'une scène avec l'ensemble des couleurs

__Q13. Visualisation des couleurs :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer une image illustrant l'ensemble des couleurs nommées.
</div>

In [22]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_all_colors.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="all_colors.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 13. <a href="all_colors.png">all_colors.png</a></i></td>
</tr></table>

## 6. Matériaux

__Q14. Liste de matériaux :__
<div style="background-color:#eef;padding:10px;border-radius:3px">
Procédez comme pour les couleurs, à partir de la table des matériaux ci-dessous, <a href="http://globe3d.sourceforge.net/g3d_html/gl-materials__ads.htm">obtenue</a> depuis le site du moteur Globe 3D.
<br><br>
Pour cela il faut :
<ul style="margin-top:0">
<li style="margin-top:0"> créer une table nommée _material_ dans la base de donnée, comportant les champs <i>id, name, phong</i>, et les champs pour les trois composantes de <i>ambient, diffuse, specular</i> et _mirror,_
<li style="margin-top:0"> alimenter la table à partir de la liste de matériaux ci-dessous, en notant que nous n'utilisons pas l'attribut nommé _emission_ et que l'attribut _shininess_ permet éventuellement de construire nos informations _phong_ et _mirror,_ en adaptant les échelles...
<li style="margin-top:0"> écrire un module nommé <code style="background-color:#eef">raytracer_materials</code> permettant d'importer les divers matériaux ainsi qu'un dictionnaire nommé _materials,_
<li style="margin-top:0"> concevoir une scène permettant de tester tout cela, et générer l'image correspondante.
</ul>
</div>

In [23]:
# Liste de matériaux

materials = {
  "Red"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (1.0, 0.0, 0.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Orange"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.992157, 0.513726, 0.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Yellow"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (1.0, 0.964706, 0.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Green"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.0, 1.0, 0.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Indigo"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.0980392, 0.0, 0.458824, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Blue"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.0, 0.0, 1.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Violet"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.635294, 0.0, 1.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "White"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.992157, 0.992157, 0.992157, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Black"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.0, 0.0, 0.0, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Medium_Gray"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.454902, 0.454902, 0.454902, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Light_Gray"  : {
    "ambient"  : (0.0, 0.0, 0.0, 1.0),
    "diffuse"  : (0.682353, 0.682353, 0.682353, 1.0),
    "specular" : (0.0225, 0.0225, 0.0225, 1.0),
    "emission" : (0.0, 0.0, 0.0, 1.0),
    "shininess": 12.8
  },

  "Glass"  : {
              "ambient"  : (0.0, 0.0, 0.0, 1.0),
              "diffuse"  : (0.588235, 0.670588, 0.729412, 1.0),
              "specular" : (0.9, 0.9, 0.9, 1.0),
              "emission" : (0.0, 0.0, 0.0, 1.0),
              "shininess": 96.0
            },

  "Brass"   : {
            "ambient":        (0.329412, 0.223529, 0.027451, 1.0),
            "diffuse":        (0.780392, 0.568627, 0.113725, 1.0),
            "specular":       (0.992157, 0.941176, 0.807843, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      27.8974
           },

  "Bronze"   : {
            "ambient":        (0.2125, 0.1275, 0.054, 1.0),
            "diffuse":        (0.714, 0.4284, 0.18144, 1.0),
            "specular":       (0.393548, 0.271906, 0.166721, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      25.6
          },

  "Polished_Bronze"   : {
            "ambient":        (0.25, 0.148, 0.06475, 1.0),
            "diffuse":        (0.4, 0.2368, 0.1036, 1.0),
            "specular":       (0.774597, 0.458561, 0.200621, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      76.8
          },

  "Chrome"   : {
            "ambient":        (0.25, 0.25, 0.25, 1.0),
            "diffuse":        (0.4, 0.4, 0.4, 1.0),
            "specular":       (0.774597, 0.774597, 0.774597, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      76.8
          },

  "Copper"   : {
            "ambient":        (0.19125, 0.0735, 0.0225, 1.0),
            "diffuse":        (0.7038, 0.27048, 0.0828, 1.0),
            "specular":       (0.256777, 0.137622, 0.086014, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      12.8
          },

  "Polished_Copper"   : {
            "ambient":        (0.2295, 0.08825, 0.0275, 1.0),
            "diffuse":        (0.5508, 0.2118, 0.066, 1.0),
            "specular":       (0.580594, 0.223257, 0.0695701, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      51.2
          },

  "Gold"   : {
            "ambient":        (0.24725, 0.1995, 0.0745, 1.0),
            "diffuse":        (0.75164, 0.60648, 0.22648, 1.0),
            "specular":       (0.628281, 0.555802, 0.366065, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      51.2
          },

  "Polished_Gold"   : {
            "ambient":        (0.24725, 0.2245, 0.0645, 1.0),
            "diffuse":        (0.34615, 0.3143, 0.0903, 1.0),
            "specular":       (0.797357, 0.723991, 0.208006, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      83.2
          },

  "Pewter"   : {
            "ambient":        (0.105882, 0.058824, 0.113725, 1.0),
            "diffuse":        (0.427451, 0.470588, 0.541176, 1.0),
            "specular":       (0.333333, 0.333333, 0.521569, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      9.84615
          },

  "Silver"   : {
            "ambient":        (0.19225, 0.19225, 0.19225, 1.0),
            "diffuse":        (0.50754, 0.50754, 0.50754, 1.0),
            "specular":       (0.508273, 0.508273, 0.508273, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      51.2
          },

  "Polished_Silver"   : {
            "ambient":        (0.23125, 0.23125, 0.23125, 1.0),
            "diffuse":        (0.2775, 0.2775, 0.2775, 1.0),
            "specular":       (0.773911, 0.773911, 0.773911, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      89.6
          },

  "Emerald"   : {
            "ambient":        (0.0215, 0.1745, 0.0215, 0.55),
            "diffuse":        (0.07568, 0.61424, 0.07568, 0.55),
            "specular":       (0.633, 0.727811, 0.633, 0.55),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      76.8
          },

  "Jade"   : {
            "ambient":        (0.135, 0.2225, 0.1575, 0.95),
            "diffuse":        (0.54, 0.89, 0.63, 0.95),
            "specular":       (0.316228, 0.316228, 0.316228, 0.95),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      12.8
          },

  "Obsidian"   : {
            "ambient":        (0.05375, 0.05, 0.06625, 0.82),
            "diffuse":        (0.18275, 0.17, 0.22525, 0.82),
            "specular":       (0.332741, 0.328634, 0.346435, 0.82),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      38.4
          },

  "Pearl"   : {
            "ambient":        (0.25, 0.20725, 0.20725, 0.922),
            "diffuse":        (1.0, 0.829, 0.829, 0.922),
            "specular":       (0.296648, 0.296648, 0.296648, 0.922),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      11.264
          },

  "Ruby"   : {
            "ambient":        (0.1745, 0.01175, 0.01175, 0.55),
            "diffuse":        (0.61424, 0.04136, 0.04136, 0.55),
            "specular":       (0.727811, 0.626959, 0.626959, 0.55),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      76.8
          },

  "Turquoise"   : {
            "ambient":        (0.1, 0.18725, 0.1745, 0.8),
            "diffuse":        (0.396, 0.74151, 0.69102, 0.8),
            "specular":       (0.297254, 0.30829, 0.306678, 0.8),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      12.8
          },

  "Black_Plastic"   : {
            "ambient":        (0.0, 0.0, 0.0, 1.0),
            "diffuse":        (0.01, 0.01, 0.01, 1.0),
            "specular":       (0.50, 0.50, 0.50, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      32.0
          },

  "Black_Rubber"   : {
            "ambient":        (0.02, 0.02, 0.02, 1.0),
            "diffuse":        (0.01, 0.01, 0.01, 1.0),
            "specular":       (0.4, 0.4, 0.4, 1.0),
            "emission":       (0.0,0.0,0.0,0.0),
            "shininess":      10.0
          },

  "Defaults"   : {
            "ambient":        (0.2, 0.2, 0.2, 1.0),
            "diffuse":        (0.8, 0.8, 0.8, 1.0),
            "specular":       (0.0, 0.0, 0.0, 1.0),
            "emission":       (0.0, 0.0, 0.0, 1.0),
            "shininess":       25.6
          }
}

In [24]:
# votre code ici

In [25]:
# votre code ici

In [26]:
# votre code ici

<table style="border-color:transparent; border-spacing:50px 0; border-collapse:separate"><tr>
<td style="border:none"><img src="ref_all_materials.png" width="221"><br>
<center><i>exemple de résultat</i></center>
</td>
<td style="border:none"><img src="all_materials.png" width="221"><br>
<center><i>votre résultat</i></center>
</td>
</tr><tr>
<td style="border:none; text-align:center" colspan="2"><i>question 14. <a href="all_materials.png">all_materials.png</a></i></td>
</tr></table>

In [27]:
import sqlite3
conn = sqlite3.connect('raytracing.sqlite')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS color")
c.execute("DROP TABLE IF EXISTS material")
c.execute("DELETE FROM scene")
conn.commit()